In [1]:
import collections
import re
import pandas as pd

dictF = collections.OrderedDict()
anotador = 0
cont = 1
str1 = ""

for filename in ['Clear_07_3_ISO_Andressa_Bressane.xml','Clear_07_3_ISO_Eric_Eltz.xml', 'Clear_07_3_ISO_Keli_Schell.xml' ]:
    
    with open(filename,'r') as file:
        cont = 1
        for line in file:

            if line.find("EM-ID") >= 0 and line.find("/EM") >= 0:
                str1 = re.findall('(?<=>).*?(?=<)', line)[0] + " - " + str(cont)
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
                #print(str1)
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
            
            elif line.find("EM-ID") >= 0:
                str1 = re.findall('(?<=>).*?(?=$)', line)[0]
                str1 = str1.replace("\n", "")
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
            
            elif line.find("/EM") >= 0:
                str1 = str1 + " " + re.findall('(?<=^).*?(?=<)', line)[0] + " - " + str(cont)
                str1 = str1.replace("\n", "")
                print(str1)
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
                str1 = ""
            
            else:
                str1 = str1 + " " + line
                
            cont += 1
        
    anotador += 1
    
#print(len(dictF))
    
kappa_pd = pd.DataFrame.from_dict(dictF,orient='index')
kappa_pd

Bacia de Irecê - 13
Cráton do São Francisco - 29
cinturão epidérmico de antepaís - 60
cinturão epidérmico de antepaís - 109
Bacia de Irecê - 113
Cráton do São Francisco - 143
dobras e falhas E-W - 162
falhas direcionais - 176
Cinturão de Antepaís - 229
Bacia de Irecê - 233
unidades carbonáticas - 278
Grupo Una - 281
substrato quartzítico - 285
Supergrupo Espinhaço - 288
Bacia de Irecê - 294
Cinturão epidérmico de antepaís - 299
Bacia de Irecê - 324
Cráton do São Francisco - 351
Grupo Una - 399
cinturão epidérmico de antepaís - 408
cinturão epidérmico de antepaís - 473
Cráton do São Francisco - 503
cinturão epidérmico - 566
cinturão epidérmico de antepaís - 637
Bacia de Irecê - 646
Cráton do São Francisco - 702
Bacia de Irecê - 712
Cráton do São Francisco - 721
Supergrupo Espinhaço - 737
Grupo Una - 741
Supergrupo Espinhaço - 766
Bacia de Irecê - 780
Grupo Una - 828
Supergrupo Espinhaço - 841
discordância erosiva - 847
Bacia de Irecê - 859
formações Bebedouro e Salitre - 867
Formação Be

,0,1,2
Bacia de Irecê - 13,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
neoproterozoicas - 21,era,eon,era
Cráton do São Francisco - 29,CONTEXTOgeologicoDeBACIA,unidadeESTRATIGRAFICA,
cinturão epidérmico de antepaís - 60,CONTEXTOgeologicoDeBACIA,,
cinturão epidérmico de antepaís - 109,CONTEXTOgeologicoDeBACIA,,
Bacia de Irecê - 113,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
Cráton do São Francisco - 143,CONTEXTOgeologicoDeBACIA,unidadeESTRATIGRAFICA,
dobras e falhas E-W - 162,outro,,
bacia - 167,baciaSEDIMENTAR,,
falhas direcionais - 176,outro,,CONTEXTOgeologicoDeBACIA


In [2]:
classes = set()

for x in kappa_pd[0].unique():
    classes.add(x.lower())
    
for x in kappa_pd[1].unique():
    classes.add(x.lower())
    
for x in kappa_pd[2].unique():
    classes.add(x.lower())

classes

{'',
 'baciasedimentar',
 'contextogeologicodebacia',
 'eon',
 'era',
 'idade',
 'outro',
 'periodo',
 'sedimentarescarbonaticas',
 'sedimentaresquimicas',
 'sedimentaressiliciclasticas',
 'unidadeestratigrafica'}

In [3]:
w, h = len(classes), len(kappa_pd);
kappa_table = [[0 for x in range(w)] for y in range(h)] 

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    for clas in classes:
        if row[0].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_table[row_id][class_id] += 1
        class_id += 1
    row_id += 1
            
kappa_table

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
 [0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
 [1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0,

In [4]:
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(kappa_table)

0.16027657450019131